### Imports

In [ ]:
import serial
import struct
import subprocess
import time
import sys
from tqdm.notebook import trange, tqdm

### Setting up environment and commands

In [ ]:
try:
    ser = serial.Serial("/dev/tty.usbmodem0000000000001")
except:
    ser = serial.Serial("/dev/ttyACM0")


def set_pulse(pulse):
    ser.write(b"\x42" + struct.pack("<I", pulse))
def set_delay(delay):
    ser.write(b"\x41" + struct.pack("<I", delay))
def glitch():
    ser.write(b"\x43")
def powerup():
    ser.write(b"\x44")
def powerdown():
    ser.write(b"\x45")

### Function that tests for succesful debug swd connection

In [ ]:
def test_swd():
    try:
        subprocess.check_output(['openocd', '-f', 'interface/jlink.cfg', '-c', 'transport select swd', '-f', 'testnrf.cfg', '-c', 'init;dump_image partial_test_dump.bin 0x0 0x1000; exit'], stderr=subprocess.STDOUT)
        return True
    except:
        pass
    return False

### Test Powerup of Airtag

In [ ]:
powerup()

### Test Powerdown of Airtag

In [ ]:
powerdown()

## Run the Glitch!

Adjust the range of Delay accordingly

In [ ]:
while True:
    for delay in trange(72500, 74000):
        set_delay(delay)
        for pulse in range(7, 9):
            set_pulse(pulse)
            glitch()
            time.sleep(0.05)
            if test_swd():
                print("Glitch successful - swd debug activated (APProtect disabled until powercycle)")
                sys.exit(0)